In [1]:
import BhrLgcGPTProcess
import BhrLgcManualProcess

import pandas as pd
import numpy as np
import json
import re
import pickle
import hashlib

import json

In [2]:
java_json = """

{"world": {"time": 1753686408125}, "npcs": [{"npcId": 10008, "status": "move", "selling": [], "items": [], "action": {"actionName": "move", "actionId": 112, "param": {"gridX": 0.0, "gridY": 0.0, "npcId": 10010.0}}, "curAction": {}, "mapData": [], "surroundings": {"people": [{"npcId": 10006, "status": "free"}, {"npcId": 10007, "status": "Fishing"}, {"npcId": 10009, "status": "free"}, {"npcId": 10010, "status": "move"}], "items": []}, "talk": {"isTalking": false}}]}
"""

In [3]:
test_json = json.loads(java_json)
# if "mapObj" in test_json:
#     del test_json["mapObj"]
test_json

{'world': {'time': 1753686408125},
 'npcs': [{'npcId': 10008,
   'status': 'move',
   'selling': [],
   'items': [],
   'action': {'actionName': 'move',
    'actionId': 112,
    'param': {'gridX': 0.0, 'gridY': 0.0, 'npcId': 10010.0}},
   'curAction': {},
   'mapData': [],
   'surroundings': {'people': [{'npcId': 10006, 'status': 'free'},
     {'npcId': 10007, 'status': 'Fishing'},
     {'npcId': 10009, 'status': 'free'},
     {'npcId': 10010, 'status': 'move'}],
    'items': []},
   'talk': {'isTalking': False}}]}

In [12]:
def parse_isFindingPeopletoTalk(json_input):
    """
    Checks if the NPC is currently performing actionId = 112 (finding people to talk).
    Returns True if yes, False otherwise.
    """
    try:
        data = json.loads(json_input)
    except json.JSONDecodeError as e:
        result = f"Error parsing JSON: {e}"
        print("Method: parse_isFindingPeopletoTalk | Description: Checks if NPC is looking for people to talk (actionId=112) | Result:", result, "\n")
        return result, None
    
    npcs = data.get('npcs', [])
    if not npcs:
        result = False
        print("Method: parse_isFindingPeopletoTalk | Description: Checks if NPC is looking for people to talk | Result:", result, "\n")
        return result, None
    
    npc = npcs[0]
    cur_action = npc.get('action', {})
    action_id = cur_action.get('actionId', '0')
    targetNPCId = cur_action.get('param', {}).get('npcId', None)
    print(action_id, targetNPCId)
    if targetNPCId:
        result = (int(action_id) == 112)
    else:
        result = False
    print("Method: parse_isFindingPeopletoTalk | Description: Checks if NPC is looking for people to talk | Result:", result, "\n")

    return result, targetNPCId

parse_isFindingPeopletoTalk(java_json)

112 10010.0
Method: parse_isFindingPeopletoTalk | Description: Checks if NPC is looking for people to talk | Result: True 



(True, 10010.0)

In [9]:
talkingInfo, is_talking = BhrLgcManualProcess.parse_talking_from_java(java_json)
is_idling = BhrLgcManualProcess.parse_isIdling(java_json)
    

inputInHumanString = BhrLgcManualProcess.parse_npc_info_for_nextaction(java_json)

is_findingToTalk = BhrLgcManualProcess.parse_isFindingPeopletoTalk(java_json) # Last action is finding people to talk
is_idling = BhrLgcManualProcess.parse_isIdling(java_json)
is_talking = BhrLgcManualProcess.parse_isTalking(java_json) 
is_buying = BhrLgcManualProcess.parse_isBuying(java_json)

target_sleeping, sleep_target_name = BhrLgcManualProcess.parse_target_sleeping(java_json) 
target_talking, talk_target_name = BhrLgcManualProcess.parse_target_talking(java_json)
shop_target_present, shopowner_target_name  = BhrLgcManualProcess.parse_target_oid_owner_at_shop(java_json)


to_memory = BhrLgcManualProcess.parse_npc_info_formemory(java_json)

print("talkingInfo:", talkingInfo)
print("is_talking:", is_talking)
print("is_idling:", is_idling)
print("inputInHumanString:", inputInHumanString)
print("is_findingToTalk:", is_findingToTalk)
print("is_talking:", is_talking)
print("is_buying:", is_buying)
print("target_sleeping:", target_sleeping)
print("sleep_target_name:", sleep_target_name)
print("target_talking:", target_talking)
print("talk_target_name:", talk_target_name)
print("shop_target_present:", shop_target_present)
print("shopowner_target_name:", shopowner_target_name)
print("to_memory:", to_memory)

NameError: name 'BhrLgcManualProcess' is not defined

In [ ]:
test_json

Traceback (most recent call last):
  File "/Users/jackhan/.vscode/extensions/ms-python.python-2024.22.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'test_json' is not defined



In [ ]:
import sys
import os
# Add the base directory (one level up from the current directory)
base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(base_dir)

from DBConnect import DBCon
from DBConnect import BhrDBJavaBuffer
from DBConnect import BhrDBInstruction
from DBConnect import BhrDBReflectionTracer
from DBConnect import BhrDBMemStre
from DBConnect import BhrDBReflection
from DBConnect import BhrDBSchedule

import BhrLgcGPTProcess
import BhrLgcManualProcess
import BhrLgcToMemStre

import pandas as pd
import numpy as np
import json
import re
import pickle
import hashlib
import configparser
import yaml

config = configparser.ConfigParser()
# Adjust path to look for config.ini in AImodule regardless of the current directory
base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
config_path = os.path.join(base_dir, 'config.ini')
config.read(config_path)

yaml_path = os.path.join(base_dir, 'char_config.yaml')

# Load the YAML file

with open(yaml_path, 'r', encoding='utf-8' ) as file:
    char_config = yaml.safe_load(file)
    print("YAML content loaded successfully.")


Traceback (most recent call last):
  File "/Users/jackhan/.vscode/extensions/ms-python.python-2024.22.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 4, in <module>
NameError: name '__file__' is not defined. Did you mean: '__name__'?



In [ ]:

npc = next((npc for npc in char_config['npcCharacters'] if npc['npcId'] == npcId), None)
if not npc:
    raise ValueError(f"NPC with npcId {npcId} not found in char.yaml")
cur_schedule_str =  npc['schedule']

Traceback (most recent call last):
  File "/Users/jackhan/.vscode/extensions/ms-python.python-2024.22.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2, in <module>
NameError: name 'char_config' is not defined

